# Hybrid Control Demo (Self-contained)
*No external imports required. This notebook defines FSM × PID × LLM mock inline and runs the demo.*

In [ ]:
# --- Inline: FSM × PID × LLM mock simulator ---
import time, random

class FSM_PID_LLM_Simulator:
    def __init__(self):
        self.states = ['IDLE', 'MOVE', 'AVOID', 'CHARGE']
        self.current_state = 'IDLE'
        self.logs = []

    def pid_control(self, setpoint, measurement):
        error = setpoint - measurement
        return 0.5 * error  # simple P control

    def llm_decision(self, state, context):
        if "battery low" in context:
            return 'CHARGE'
        elif "obstacle" in context:
            return 'AVOID'
        elif "task start" in context:
            return 'MOVE'
        else:
            return 'IDLE'

    def step(self, t):
        context = random.choice(["normal", "battery low", "obstacle ahead", "task start"])
        next_state = self.llm_decision(self.current_state, context)
        log = {"time": t, "state": self.current_state, "context": context, "llm_suggest": next_state}
        if next_state != self.current_state:
            log["transition"] = f"{self.current_state} → {next_state}"
            self.current_state = next_state
        if self.current_state == 'MOVE':
            setpoint = 10.0
            measurement = random.uniform(7.0, 12.0)
            control = self.pid_control(setpoint, measurement)
            log["pid"] = {"target": setpoint, "measured": measurement, "output": control}
        elif self.current_state == 'AVOID':
            log["action"] = "Avoiding obstacle"
        elif self.current_state == 'CHARGE':
            log["action"] = "Charging"
        else:
            log["action"] = "System idle"
        self.logs.append(log)
        return log

In [ ]:
# --- Inline: GoalReasoningAgent ---
from dataclasses import dataclass, field
from typing import Dict

@dataclass
class GoalReasoningAgent:
    initial_goal: str = "IDLE"
    verbose: bool = True
    rules: Dict[str, str] = field(default_factory=lambda: {
        "battery low": "CHARGE",
        "obstacle": "AVOID",
        "destination": "NAVIGATE",
        "task start": "MOVE",
    })

    def __post_init__(self):
        self.current_goal = self.initial_goal

    def decide_next_goal(self, observation_text: str):
        obs_lc = (observation_text or "").lower()
        matched, next_goal = None, self.current_goal
        for key, goal in self.rules.items():
            if key in obs_lc:
                matched, next_goal = key, goal
                break
        changed = (next_goal != self.current_goal)
        if self.verbose:
            print(f"🧠 Observation: {observation_text}")
            print(f"🔎 Matched rule: {matched}" if matched else "ℹ️  No rule matched")
            print(f"🎯 Goal updated: {self.current_goal} → {next_goal}" if changed else f"✔️ Goal remains: {self.current_goal}")
        self.current_goal = next_goal
        return next_goal, {"matched_rule": matched, "changed": changed}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

sim = FSM_PID_LLM_Simulator()
logs = [sim.step(t) for t in range(40)]
df = pd.json_normalize(logs).fillna('')
df.head()

In [ ]:
# Show last rows
df.tail(10)

In [ ]:
# Plot state over time
state_to_idx = {s:i for i,s in enumerate(['IDLE','MOVE','AVOID','CHARGE'])}
y = [state_to_idx.get(s, -1) for s in df['state']]
x = df['time']

plt.figure()
plt.step(x, y, where='post')
plt.yticks(list(state_to_idx.values()), list(state_to_idx.keys()))
plt.xlabel("time step")
plt.ylabel("state")
plt.title("FSM State over Time (Self-contained)")
plt.show()

In [ ]:
# GoalReasoningAgent demo
agent = GoalReasoningAgent(initial_goal="IDLE", verbose=True)
samples = ["task start received", "obstacle near", "battery low", "destination ahead"]
for obs in samples:
    goal, meta = agent.decide_next_goal(obs)
    print({"obs": obs, "goal": goal, **meta})